<br>
@Author: Ayush Prajapati<br>
@Date: 13-09-2024<br>
@Last Modified by: Ayush Prajapati<br>
@Last Modified time: 13-09-2024<br>
@Title: S3 CRUD operations using Boto3 Library on SQL Server (AWS RDS)<br>

In [1]:
import boto3
from dotenv import load_dotenv
from io import StringIO
import pandas as pd
import pyodbc
import os

In [2]:
load_dotenv()

True

## Configuring boto3 with s3

**Create a boto3 client**

In [3]:
s3 = boto3.client('s3')

**List all buckets**

In [4]:
response = s3.list_buckets()

# Output bucket names
for bucket in response['Buckets']:
    print(bucket['Name'])

ayush-dataset-rds


## Operations

In [5]:
def upload_csv_file_to_s3(bucket_name, file_name, s3_key):
    """
    Description:
        Upload a CSV file to an S3 bucket in AWS
    Parameters:
        bucket_name = name of bucket
        file_name = local file name (path)
        s3_key = name of the file in S3
    Returns:
        None
    """
    try:
        s3.upload_file(
            file_name, 
            bucket_name, 
            s3_key, 
            ExtraArgs={'ContentType': 'text/csv'}
        )
        print(f"File {file_name} uploaded successfully to {bucket_name}/{s3_key} as CSV.")
    except Exception as e:
        print(f"Error uploading file: {e}")


In [6]:
def main():
    bucket_name = 'ayush-dataset-rds'
    csv_file_name = 'worldometer_data.csv'
    s3_key = 'csv/worldometer_data.csv'

    upload_csv_file_to_s3(bucket_name, csv_file_name, s3_key)


if __name__ == '__main__':
    main()

File worldometer_data.csv uploaded successfully to ayush-dataset-rds/csv/worldometer_data.csv as CSV.


**Fetching file from S3 Bucket**

In [59]:
bucket_name = 'ayush-dataset-rds'
s3_key = 'csv/worldometer_data.csv'

try:
    # Fetch the file from S3
    obj = s3.get_object(Bucket=bucket_name, Key=s3_key)
    csv_content = obj['Body'].read().decode('utf-8')

    # Use StringIO to treat the string as a file
    csv_string_io = StringIO(csv_content)

    df = pd.read_csv(csv_string_io)
    print("CSV content:")
    display(df)
        
except Exception as e:
    print(f"Error reading file from S3: {e}")


CSV content:


,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region
0,USA,North America,3.311981e+08,5032179,NaN,162804.0,NaN,2576668.0,NaN,2292707.0,18296.0,15194.0,492.0,63139605.0,190640.0,Americas
1,Brazil,South America,2.127107e+08,2917562,NaN,98644.0,NaN,2047660.0,NaN,771258.0,8318.0,13716.0,464.0,13206188.0,62085.0,Americas
2,India,Asia,1.381345e+09,2025409,NaN,41638.0,NaN,1377384.0,NaN,606387.0,8944.0,1466.0,30.0,22149351.0,16035.0,South-EastAsia
3,Russia,Europe,1.459409e+08,871894,NaN,14606.0,NaN,676357.0,NaN,180931.0,2300.0,5974.0,100.0,29716907.0,203623.0,Europe
4,South Africa,Africa,5.938157e+07,538184,NaN,9604.0,NaN,387316.0,NaN,141264.0,539.0,9063.0,162.0,3149807.0,53044.0,Africa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,Montserrat,North America,4.992000e+03,13,NaN,1.0,NaN,10.0,NaN,2.0,NaN,2604.0,200.0,61.0,12220.0,NaN
205,Caribbean Netherlands,North America,2.624700e+04,13,NaN,NaN,NaN,7.0,NaN,6.0,NaN,495.0,NaN,424.0,16154.0,NaN
206,Falkland Islands,South America,3.489000e+03,13,NaN,NaN,NaN,13.0,NaN,0.0,NaN,3726.0,NaN,1816.0,520493.0,NaN
207,Vatican City,Europe,8.010000e+02,12,NaN,NaN,NaN,12.0,NaN,0.0,NaN,14981.0,NaN,NaN,NaN,Europe


In [48]:

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209 entries, 0 to 208
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Country/Region    209 non-null    object 
 1   Continent         208 non-null    object 
 2   Population        208 non-null    float64
 3   TotalCases        209 non-null    int64  
 4   NewCases          4 non-null      float64
 5   TotalDeaths       188 non-null    float64
 6   NewDeaths         3 non-null      float64
 7   TotalRecovered    205 non-null    float64
 8   NewRecovered      3 non-null      float64
 9   ActiveCases       205 non-null    float64
 10  Serious,Critical  122 non-null    float64
 11  Tot Cases/1M pop  208 non-null    float64
 12  Deaths/1M pop     187 non-null    float64
 13  TotalTests        191 non-null    float64
 14  Tests/1M pop      191 non-null    float64
 15  WHO Region        184 non-null    object 
dtypes: float64(12), int64(1), object(3)
memory u

**Cleaning Data**

In [2]:
def clean_data(df):
    """
    Clean the DataFrame by filling null values with appropriate defaults,
    removing duplicates, and formatting columns.
    Parameters:
        df: DataFrame to be cleaned
    Returns:
        Cleaned DataFrame
    """
    # Fill missing values with suitable defaults
    df['Continent'].fillna('Unknown', inplace=True)
    df['Population'].fillna(0, inplace=True)  # Assuming population can't be negative
    df['TotalDeaths'].fillna(0, inplace=True)
    df['NewCases'].fillna(0, inplace=True)
    df['TotalRecovered'].fillna(0, inplace=True)
    df['NewRecovered'].fillna(0, inplace=True)
    df['ActiveCases'].fillna(0, inplace=True)
    df['Serious,Critical'].fillna(0, inplace=True)
    df['Tot Cases/1M pop'].fillna(0, inplace=True)
    df['Deaths/1M pop'].fillna(0, inplace=True)
    df['TotalTests'].fillna(0, inplace=True)
    df['Tests/1M pop'].fillna(0, inplace=True)
    df['WHO Region'].fillna('Unknown', inplace=True)

    # Convert relevant columns to numeric, coercing errors to NaN
    numeric_columns = [
        'Population', 'TotalCases', 'NewCases', 'TotalDeaths',
        'NewDeaths', 'TotalRecovered', 'NewRecovered', 'ActiveCases',
        'Serious,Critical', 'Tot Cases/1M pop', 'Deaths/1M pop',
        'TotalTests', 'Tests/1M pop'
    ]
    
    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Fill NaN values after conversion with appropriate defaults
    for col in numeric_columns:
        df[col].fillna(0, inplace=True)

    # Remove duplicates based on all columns
    df.drop_duplicates(inplace=True)

    # Reset index for cleanliness
    df.reset_index(drop=True, inplace=True)

    return df


In [3]:
df = pd.read_csv('worldometer_data.csv')

**Clean the data**

In [4]:

df_cleaned = clean_data(df)

print("Cleaned CSV content:")
display(df_cleaned)

Cleaned CSV content:


,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region
0,USA,North America,3.311981e+08,5032179,0.0,162804.0,0.0,2576668.0,0.0,2292707.0,18296.0,15194.0,492.0,63139605.0,190640.0,Americas
1,Brazil,South America,2.127107e+08,2917562,0.0,98644.0,0.0,2047660.0,0.0,771258.0,8318.0,13716.0,464.0,13206188.0,62085.0,Americas
2,India,Asia,1.381345e+09,2025409,0.0,41638.0,0.0,1377384.0,0.0,606387.0,8944.0,1466.0,30.0,22149351.0,16035.0,South-EastAsia
3,Russia,Europe,1.459409e+08,871894,0.0,14606.0,0.0,676357.0,0.0,180931.0,2300.0,5974.0,100.0,29716907.0,203623.0,Europe
4,South Africa,Africa,5.938157e+07,538184,0.0,9604.0,0.0,387316.0,0.0,141264.0,539.0,9063.0,162.0,3149807.0,53044.0,Africa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,Montserrat,North America,4.992000e+03,13,0.0,1.0,0.0,10.0,0.0,2.0,0.0,2604.0,200.0,61.0,12220.0,Unknown
205,Caribbean Netherlands,North America,2.624700e+04,13,0.0,0.0,0.0,7.0,0.0,6.0,0.0,495.0,0.0,424.0,16154.0,Unknown
206,Falkland Islands,South America,3.489000e+03,13,0.0,0.0,0.0,13.0,0.0,0.0,0.0,3726.0,0.0,1816.0,520493.0,Unknown
207,Vatican City,Europe,8.010000e+02,12,0.0,0.0,0.0,12.0,0.0,0.0,0.0,14981.0,0.0,0.0,0.0,Europe


**Connection with the RDS Database**

In [24]:
server = os.getenv('db_server')
USERNAME = os.getenv('db_username')
PASSWORD = os.getenv('db_password')


try:
    connectionString = (
        f"""DRIVER={{ODBC Driver 18 for SQL Server}};
        SERVER={server};
        UID={USERNAME};
        PWD={PASSWORD};
        TrustServerCertificate=yes;"""
    )
    connection = pyodbc.connect(connectionString, autocommit=True)
    print("Connected to the database successfully!")

except pyodbc.Error as e:
    print(f"Error connecting to database: {e}")


Connected to the database successfully!


In [25]:
cursor = connection.cursor()

**Connecting to Database**

In [28]:
try:
    db_query = "USE ayushdemo;"
    cursor.execute(db_query)
    print("Using the database")
except pyodbc.Error as e:
        print(f"Error connecting to database: {e}")

Using the database


**Creating a table**

In [32]:
def create_table(connection):
    """
    Create a table in the database to store CSV data.
    Parameters:
        connection: active database connection
    Returns:
        None
    """
    cursor = connection.cursor()
    
    create_table_query = """
    CREATE TABLE worldometer (
        CountryRegion NVARCHAR(100),
        Population FLOAT,
        TotalCases INT,
        NewCases FLOAT,
        TotalDeaths FLOAT,
        NewDeaths FLOAT,
        TotalRecovered FLOAT,
        NewRecovered FLOAT,
        ActiveCases FLOAT,
        SeriousCritical FLOAT,
        TotCasesPer1M FLOAT,
        DeathsPer1M FLOAT,
        TotalTests FLOAT,
        TestsPer1M FLOAT,
        WHORegion NVARCHAR(100)
    )
    """
    
    try:
        cursor.execute(create_table_query)
        connection.commit()
        print("Table created successfully!")
    except Exception as e:
        print(f"Error creating table: {e}")
    finally:
        cursor.close()


In [33]:
create_table(connection)

Table created successfully!


**Inserting Data**

In [34]:
def insert_data_to_db(connection, df):
    """
    Insert the cleaned DataFrame into the database table.
    Parameters:
        connection: active database connection
        df: cleaned DataFrame
    Returns:
        None
    """
    cursor = connection.cursor()
    
    # Prepare the insert query
    insert_query = """
    INSERT INTO worldometer (
        CountryRegion, Population, TotalCases, NewCases, TotalDeaths,
        NewDeaths, TotalRecovered, NewRecovered, ActiveCases, 
        SeriousCritical, TotCasesPer1M, DeathsPer1M,
        TotalTests, TestsPer1M, WHORegion
    )
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """

    # Execute the insert query for each row in the DataFrame
    for index, row in df.iterrows():
        cursor.execute(insert_query, 
                       row['Country/Region'], 
                       row['Population'], 
                       row['TotalCases'], 
                       row['NewCases'], 
                       row['TotalDeaths'], 
                       row['NewDeaths'], 
                       row['TotalRecovered'], 
                       row['NewRecovered'], 
                       row['ActiveCases'], 
                       row['Serious,Critical'], 
                       row['Tot Cases/1M pop'], 
                       row['Deaths/1M pop'], 
                       row['TotalTests'], 
                       row['Tests/1M pop'], 
                       row['WHO Region']
                      )

In [62]:
insert_data_to_db(connection, df_cleaned)

**Reading Data**

In [63]:
def read_data(cursor):
    select_query = "SELECT * FROM worldometer"

    cursor.execute(select_query)
    rows = cursor.fetchall()

    if rows:
        print("--------------------------------------------------------")
        for row in rows:
            print(row)
    else:
        print("No data found in the 'employees' table.")

In [64]:
read_data(cursor)

--------------------------------------------------------
('USA', 331198130.0, 5032179, 0.0, 162804.0, 0.0, 2576668.0, 0.0, 2292707.0, 18296.0, 15194.0, 492.0, 63139605.0, 190640.0, 'Americas')
('Brazil', 212710692.0, 2917562, 0.0, 98644.0, 0.0, 2047660.0, 0.0, 771258.0, 8318.0, 13716.0, 464.0, 13206188.0, 62085.0, 'Americas')
('India', 1381344997.0, 2025409, 0.0, 41638.0, 0.0, 1377384.0, 0.0, 606387.0, 8944.0, 1466.0, 30.0, 22149351.0, 16035.0, 'South-EastAsia')
('Russia', 145940924.0, 871894, 0.0, 14606.0, 0.0, 676357.0, 0.0, 180931.0, 2300.0, 5974.0, 100.0, 29716907.0, 203623.0, 'Europe')
('South Africa', 59381566.0, 538184, 0.0, 9604.0, 0.0, 387316.0, 0.0, 141264.0, 539.0, 9063.0, 162.0, 3149807.0, 53044.0, 'Africa')
('Mexico', 129066160.0, 462690, 6590.0, 50517.0, 819.0, 308848.0, 4140.0, 103325.0, 3987.0, 3585.0, 391.0, 1056915.0, 8189.0, 'Americas')
('Peru', 33016319.0, 455409, 0.0, 20424.0, 0.0, 310337.0, 0.0, 124648.0, 1426.0, 13793.0, 619.0, 2493429.0, 75521.0, 'Americas')
('C

**Performing Queries**

In [66]:
query1 = """
SELECT 
    SUM(CAST(Population AS BIGINT)) AS Global_Population,
    SUM(TotalCases) AS TotalCases,
    (((SUM(TotalCases) * 100.0)/ SUM(CAST(Population AS BIGINT)))) AS Global_Infection_Rate
FROM 
    worldometer;

"""

cursor.execute(query1)
rows = cursor.fetchall()

# Display the results in a readable format
for row in rows:
    print(f"Global Population: {row.Global_Population}, Total Cases: {row.TotalCases}, Global Infection Rate: {row.Global_Infection_Rate:.2f}%")


Global Population: 6326421290, Total Cases: 19169166, Global Infection Rate: 0.30%


In [73]:
query2 = """
SELECT 
    CountryRegion, 
    SUM(TotalDeaths) AS Total_Continent_Deaths
FROM worldometer
GROUP BY CountryRegion
ORDER BY Total_Continent_Deaths DESC;
"""

print("\nCountries with the highest infection rates:")
cursor.execute(query2)
rows = cursor.fetchall()

for row in rows:
    print(row)


Countries with the highest infection rates:
('USA', 162804.0)
('Brazil', 98644.0)
('Mexico', 50517.0)
('UK', 46413.0)
('India', 41638.0)
('Italy', 35187.0)
('France', 30312.0)
('Spain', 28500.0)
('Peru', 20424.0)
('Iran', 17976.0)
('Russia', 14606.0)
('Colombia', 11939.0)
('Chile', 9889.0)
('Belgium', 9859.0)
('South Africa', 9604.0)
('Germany', 9252.0)
('Canada', 8966.0)
('Netherlands', 6153.0)
('Pakistan', 6035.0)
('Ecuador', 5877.0)
('Turkey', 5798.0)
('Sweden', 5766.0)
('Indonesia', 5521.0)
('Iraq', 5161.0)
('Egypt', 4951.0)
('Argentina', 4251.0)
('Bolivia', 3465.0)
('Bangladesh', 3306.0)
('Saudi Arabia', 3055.0)
('Romania', 2566.0)
('Philippines', 2150.0)
('Guatemala', 2119.0)
('Switzerland', 1985.0)
('Ukraine', 1819.0)
('Poland', 1774.0)
('Ireland', 1768.0)
('Portugal', 1743.0)
('Panama', 1574.0)
('Kyrgyzstan', 1447.0)
('Honduras', 1423.0)
('Afghanistan', 1298.0)
('Algeria', 1273.0)
('Dominican Republic', 1246.0)
('Kazakhstan', 1058.0)
('Japan', 1026.0)
('Nigeria', 930.0)
('Mold

**Exporting query output to csv**

In [74]:
# Read data into a DataFrame
new_df= pd.read_sql(query2, connection)

# Export DataFrame to CSV
new_df.to_csv('rds_query_data.csv', index=False)
print("rds_query_data.csv")


rds_query_data.csv


/tmp/ipykernel_14680/1669932501.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  new_df= pd.read_sql(query2, connection)


**Saving the cleaned dataset for future use**

In [8]:
df_cleaned.to_csv('worldometer_data_cleaned.csv', index=True)

In [9]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209 entries, 0 to 208
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Country/Region    209 non-null    object 
 1   Continent         209 non-null    object 
 2   Population        209 non-null    float64
 3   TotalCases        209 non-null    int64  
 4   NewCases          209 non-null    float64
 5   TotalDeaths       209 non-null    float64
 6   NewDeaths         209 non-null    float64
 7   TotalRecovered    209 non-null    float64
 8   NewRecovered      209 non-null    float64
 9   ActiveCases       209 non-null    float64
 10  Serious,Critical  209 non-null    float64
 11  Tot Cases/1M pop  209 non-null    float64
 12  Deaths/1M pop     209 non-null    float64
 13  TotalTests        209 non-null    float64
 14  Tests/1M pop      209 non-null    float64
 15  WHO Region        209 non-null    object 
dtypes: float64(12), int64(1), object(3)
memory u